In [2]:

import matplotlib.pyplot as plt
# 导入数据集生成工具
import numpy as np
import seaborn as sns
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Sequential, regularizers
from mpl_toolkits.mplot3d import Axes3D

plt.rcParams['font.size'] = 16
plt.rcParams['font.family'] = ['STKaiti']
plt.rcParams['axes.unicode_minus'] = False

OUTPUT_DIR = 'output_dir'
N_EPOCHS = 500


def load_dataset():
    # 采样点数
    N_SAMPLES = 1000
    # 测试数量比率
    TEST_SIZE = None

    # 从 moon 分布中随机采样 1000 个点，并切分为训练集-测试集
    X, y = make_moons(n_samples=N_SAMPLES, noise=0.25, random_state=100)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=42)
    return X, y, X_train, X_test, y_train, y_test


def make_plot(X, y, plot_name, file_name, XX=None, YY=None, preds=None, dark=False, output_dir=OUTPUT_DIR):
    # 绘制数据集的分布， X 为 2D 坐标， y 为数据点的标签
    if dark:
        plt.style.use('dark_background')
    else:
        sns.set_style("whitegrid")
    axes = plt.gca()
    axes.set_xlim([-2, 3])
    axes.set_ylim([-1.5, 2])
    axes.set(xlabel="$x_1$", ylabel="$x_2$")
    plt.title(plot_name, fontsize=20, fontproperties='SimHei')
    plt.subplots_adjust(left=0.20)
    plt.subplots_adjust(right=0.80)
    if XX is not None and YY is not None and preds is not None:
        plt.contourf(XX, YY, preds.reshape(XX.shape), 25, alpha=0.08, cmap=plt.cm.Spectral)
        plt.contour(XX, YY, preds.reshape(XX.shape), levels=[.5], cmap="Greys", vmin=0, vmax=.6)
    # 绘制散点图，根据标签区分颜色m=markers
    markers = ['o' if i == 1 else 's' for i in y.ravel()]
    mscatter(X[:, 0], X[:, 1], c=y.ravel(), s=20, cmap=plt.cm.Spectral, edgecolors='none', m=markers, ax=axes)
    # 保存矢量图
    plt.savefig(output_dir + '/' + file_name)
    plt.close()


def mscatter(x, y, ax=None, m=None, **kw):
    import matplotlib.markers as mmarkers
    if not ax: ax = plt.gca()
    sc = ax.scatter(x, y, **kw)
    if (m is not None) and (len(m) == len(x)):
        paths = []
        for marker in m:
            if isinstance(marker, mmarkers.MarkerStyle):
                marker_obj = marker
            else:
                marker_obj = mmarkers.MarkerStyle(marker)
            path = marker_obj.get_path().transformed(
                marker_obj.get_transform())
            paths.append(path)
        sc.set_paths(paths)
    return sc


def network_layers_influence(X_train, y_train):
    # 构建 5 种不同层数的网络
    for n in range(5):
        # 创建容器
        model = Sequential()
        # 创建第一层
        model.add(layers.Dense(8, input_dim=2, activation='relu'))
        # 添加 n 层，共 n+2 层
        for _ in range(n):
            model.add(layers.Dense(32, activation='relu'))
        # 创建最末层
        model.add(layers.Dense(1, activation='sigmoid'))
        # 模型装配与训练
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(X_train, y_train, epochs=N_EPOCHS, verbose=1)
        # 绘制不同层数的网络决策边界曲线
        # 可视化的 x 坐标范围为[-2, 3]
        xx = np.arange(-2, 3, 0.01)
        # 可视化的 y 坐标范围为[-1.5, 2]
        yy = np.arange(-1.5, 2, 0.01)
        # 生成 x-y 平面采样网格点，方便可视化
        XX, YY = np.meshgrid(xx, yy)
        preds = model.predict_classes(np.c_[XX.ravel(), YY.ravel()])
        title = "网络层数：{0}".format(2 + n)
        file = "网络容量_%i.png" % (2 + n)
        make_plot(X_train, y_train, title, file, XX, YY, preds, output_dir=OUTPUT_DIR + '/network_layers')


def dropout_influence(X_train, y_train):
    # 构建 5 种不同数量 Dropout 层的网络
    for n in range(5):
        # 创建容器
        model = Sequential()
        # 创建第一层
        model.add(layers.Dense(8, input_dim=2, activation='relu'))
        counter = 0
        # 网络层数固定为 5
        for _ in range(5):
            model.add(layers.Dense(64, activation='relu'))
        # 添加 n 个 Dropout 层
        if counter < n:
            counter += 1
            model.add(layers.Dropout(rate=0.5))

        # 输出层
        model.add(layers.Dense(1, activation='sigmoid'))
        # 模型装配
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        # 训练
        model.fit(X_train, y_train, epochs=N_EPOCHS, verbose=1)
        # 绘制不同 Dropout 层数的决策边界曲线
        # 可视化的 x 坐标范围为[-2, 3]
        xx = np.arange(-2, 3, 0.01)
        # 可视化的 y 坐标范围为[-1.5, 2]
        yy = np.arange(-1.5, 2, 0.01)
        # 生成 x-y 平面采样网格点，方便可视化
        XX, YY = np.meshgrid(xx, yy)
        preds = model.predict_classes(np.c_[XX.ravel(), YY.ravel()])
        title = "无Dropout层" if n == 0 else "{0}层 Dropout层".format(n)
        file = "Dropout_%i.png" % n
        make_plot(X_train, y_train, title, file, XX, YY, preds, output_dir=OUTPUT_DIR + '/dropout')


def build_model_with_regularization(_lambda):
    # 创建带正则化项的神经网络
    model = Sequential()
    model.add(layers.Dense(8, input_dim=2, activation='relu'))  # 不带正则化项
    # 2-4层均是带 L2 正则化项
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(_lambda)))
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(_lambda)))
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(_lambda)))
    # 输出层
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # 模型装配
    return model


def plot_weights_matrix(model, layer_index, plot_name, file_name, output_dir=OUTPUT_DIR):
    # 绘制权值范围函数
    # 提取指定层的权值矩阵
    weights = model.layers[layer_index].get_weights()[0]
    shape = weights.shape
    # 生成和权值矩阵等大小的网格坐标
    X = np.array(range(shape[1]))
    Y = np.array(range(shape[0]))
    X, Y = np.meshgrid(X, Y)
    # 绘制3D图
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    plt.title(plot_name, fontsize=20, fontproperties='SimHei')
    # 绘制权值矩阵范围
    ax.plot_surface(X, Y, weights, cmap=plt.get_cmap('rainbow'), linewidth=0)
    # 设置坐标轴名
    ax.set_xlabel('网格x坐标', fontsize=16, rotation=0, fontproperties='SimHei')
    ax.set_ylabel('网格y坐标', fontsize=16, rotation=0, fontproperties='SimHei')
    ax.set_zlabel('权值', fontsize=16, rotation=90, fontproperties='SimHei')
    # 保存矩阵范围图
    plt.savefig(output_dir + "/" + file_name + ".svg")
    plt.close(fig)


def regularizers_influence(X_train, y_train):
    for _lambda in [1e-5, 1e-3, 1e-1, 0.12, 0.13]:  # 设置不同的正则化系数
        # 创建带正则化项的模型
        model = build_model_with_regularization(_lambda)
        # 模型训练
        model.fit(X_train, y_train, epochs=N_EPOCHS, verbose=1)
        # 绘制权值范围
        layer_index = 2
        plot_title = "正则化系数：{}".format(_lambda)
        file_name = "正则化网络权值_" + str(_lambda)
        # 绘制网络权值范围图
        plot_weights_matrix(model, layer_index, plot_title, file_name, output_dir=OUTPUT_DIR + '/regularizers')
        # 绘制不同正则化系数的决策边界线
        # 可视化的 x 坐标范围为[-2, 3]
        xx = np.arange(-2, 3, 0.01)
        # 可视化的 y 坐标范围为[-1.5, 2]
        yy = np.arange(-1.5, 2, 0.01)
        # 生成 x-y 平面采样网格点，方便可视化
        XX, YY = np.meshgrid(xx, yy)
        preds = model.predict_classes(np.c_[XX.ravel(), YY.ravel()])
        title = "正则化系数：{}".format(_lambda)
        file = "正则化_%g.svg" % _lambda
        make_plot(X_train, y_train, title, file, XX, YY, preds, output_dir=OUTPUT_DIR + '/regularizers')


def main():
    X, y, X_train, X_test, y_train, y_test = load_dataset()
    # 绘制数据集分布
    make_plot(X, y, None, "月牙形状二分类数据集分布.svg")
    # 网络层数的影响
    network_layers_influence(X_train, y_train)
    # Dropout的影响
    dropout_influence(X_train, y_train)
    # 正则化的影响
    regularizers_influence(X_train, y_train)


if __name__ == '__main__':
    main()


Train on 750 samples
Epoch 1/500
750/750 [==============================] - 0s 330us/sample - loss: 0.7550 - accuracy: 0.1533
Epoch 2/500
750/750 [==============================] - 0s 30us/sample - loss: 0.7277 - accuracy: 0.2867
Epoch 3/500
750/750 [==============================] - 0s 29us/sample - loss: 0.7051 - accuracy: 0.4333
Epoch 4/500
750/750 [==============================] - 0s 28us/sample - loss: 0.6853 - accuracy: 0.5520
Epoch 5/500
750/750 [==============================] - 0s 29us/sample - loss: 0.6683 - accuracy: 0.6387
Epoch 6/500
750/750 [==============================] - 0s 28us/sample - loss: 0.6514 - accuracy: 0.6640
Epoch 7/500
750/750 [==============================] - 0s 29us/sample - loss: 0.6355 - accuracy: 0.6747
Epoch 8/500
750/750 [==============================] - 0s 32us/sample - loss: 0.6193 - accuracy: 0.6800
Epoch 9/500
750/750 [==============================] - 0s 30us/sample - loss: 0.6031 - accuracy: 0.6907
Epoch 10/500
750/750 [====================

750/750 [==============================] - 0s 27us/sample - loss: 0.2684 - accuracy: 0.8840
Epoch 79/500
750/750 [==============================] - 0s 26us/sample - loss: 0.2678 - accuracy: 0.8853
Epoch 80/500
750/750 [==============================] - 0s 26us/sample - loss: 0.2673 - accuracy: 0.8840
Epoch 81/500
750/750 [==============================] - 0s 25us/sample - loss: 0.2670 - accuracy: 0.8827
Epoch 82/500
750/750 [==============================] - 0s 23us/sample - loss: 0.2666 - accuracy: 0.8827
Epoch 83/500
750/750 [==============================] - 0s 24us/sample - loss: 0.2663 - accuracy: 0.8840
Epoch 84/500
750/750 [==============================] - 0s 24us/sample - loss: 0.2662 - accuracy: 0.8867
Epoch 85/500
750/750 [==============================] - 0s 25us/sample - loss: 0.2654 - accuracy: 0.8853
Epoch 86/500
750/750 [==============================] - 0s 24us/sample - loss: 0.2651 - accuracy: 0.8840
Epoch 87/500
750/750 [==============================] - 0s 25us/samp

750/750 [==============================] - 0s 27us/sample - loss: 0.2382 - accuracy: 0.8947
Epoch 156/500
750/750 [==============================] - 0s 25us/sample - loss: 0.2380 - accuracy: 0.8960
Epoch 157/500
750/750 [==============================] - 0s 26us/sample - loss: 0.2373 - accuracy: 0.8973
Epoch 158/500
750/750 [==============================] - 0s 25us/sample - loss: 0.2366 - accuracy: 0.8960
Epoch 159/500
750/750 [==============================] - 0s 25us/sample - loss: 0.2362 - accuracy: 0.8987
Epoch 160/500
750/750 [==============================] - 0s 25us/sample - loss: 0.2361 - accuracy: 0.8987
Epoch 161/500
750/750 [==============================] - 0s 25us/sample - loss: 0.2356 - accuracy: 0.8987
Epoch 162/500
750/750 [==============================] - 0s 25us/sample - loss: 0.2349 - accuracy: 0.8987
Epoch 163/500
750/750 [==============================] - 0s 28us/sample - loss: 0.2342 - accuracy: 0.8987
Epoch 164/500
750/750 [==============================] - 0s 

750/750 [==============================] - 0s 27us/sample - loss: 0.1975 - accuracy: 0.9173
Epoch 233/500
750/750 [==============================] - 0s 26us/sample - loss: 0.1969 - accuracy: 0.9187
Epoch 234/500
750/750 [==============================] - 0s 25us/sample - loss: 0.1965 - accuracy: 0.9173
Epoch 235/500
750/750 [==============================] - 0s 24us/sample - loss: 0.1962 - accuracy: 0.9200
Epoch 236/500
750/750 [==============================] - 0s 25us/sample - loss: 0.1956 - accuracy: 0.9173
Epoch 237/500
750/750 [==============================] - 0s 24us/sample - loss: 0.1952 - accuracy: 0.9187
Epoch 238/500
750/750 [==============================] - 0s 25us/sample - loss: 0.1945 - accuracy: 0.9213
Epoch 239/500
750/750 [==============================] - 0s 24us/sample - loss: 0.1942 - accuracy: 0.9227
Epoch 240/500
750/750 [==============================] - 0s 24us/sample - loss: 0.1936 - accuracy: 0.9213
Epoch 241/500
750/750 [==============================] - 0s 

750/750 [==============================] - 0s 29us/sample - loss: 0.1692 - accuracy: 0.9307
Epoch 310/500
750/750 [==============================] - 0s 28us/sample - loss: 0.1683 - accuracy: 0.9307
Epoch 311/500
750/750 [==============================] - 0s 25us/sample - loss: 0.1681 - accuracy: 0.9293
Epoch 312/500
750/750 [==============================] - 0s 25us/sample - loss: 0.1680 - accuracy: 0.9307
Epoch 313/500
750/750 [==============================] - 0s 25us/sample - loss: 0.1674 - accuracy: 0.9307
Epoch 314/500
750/750 [==============================] - 0s 25us/sample - loss: 0.1673 - accuracy: 0.9307
Epoch 315/500
750/750 [==============================] - 0s 25us/sample - loss: 0.1670 - accuracy: 0.9293
Epoch 316/500
750/750 [==============================] - 0s 26us/sample - loss: 0.1666 - accuracy: 0.9307
Epoch 317/500
750/750 [==============================] - 0s 26us/sample - loss: 0.1665 - accuracy: 0.9307
Epoch 318/500
750/750 [==============================] - 0s 

...

750/750 [==============================] - 0s 34us/sample - loss: 0.1130 - accuracy: 0.9520
Epoch 427/500
750/750 [==============================] - 0s 35us/sample - loss: 0.1133 - accuracy: 0.9533
Epoch 428/500
750/750 [==============================] - 0s 33us/sample - loss: 0.1134 - accuracy: 0.9533
Epoch 429/500
750/750 [==============================] - 0s 33us/sample - loss: 0.1133 - accuracy: 0.9520
Epoch 430/500
750/750 [==============================] - 0s 33us/sample - loss: 0.1147 - accuracy: 0.9547
Epoch 431/500
750/750 [==============================] - 0s 33us/sample - loss: 0.1167 - accuracy: 0.9560
Epoch 432/500
750/750 [==============================] - 0s 33us/sample - loss: 0.1148 - accuracy: 0.9520
Epoch 433/500
750/750 [==============================] - 0s 33us/sample - loss: 0.1136 - accuracy: 0.9547
Epoch 434/500
750/750 [==============================] - 0s 33us/sample - loss: 0.1143 - accuracy: 0.9507
Epoch 435/500
750/750 [==============================] - 0s 

750/750 [==============================] - 0s 37us/sample - loss: 0.4916 - accuracy: 0.7573
Epoch 4/500
750/750 [==============================] - 0s 38us/sample - loss: 0.4118 - accuracy: 0.7827
Epoch 5/500
750/750 [==============================] - 0s 34us/sample - loss: 0.3426 - accuracy: 0.8360
Epoch 6/500
750/750 [==============================] - 0s 33us/sample - loss: 0.2910 - accuracy: 0.8760
Epoch 7/500
750/750 [==============================] - 0s 33us/sample - loss: 0.2521 - accuracy: 0.9053
Epoch 8/500
750/750 [==============================] - 0s 33us/sample - loss: 0.2265 - accuracy: 0.9093
Epoch 9/500
750/750 [==============================] - 0s 33us/sample - loss: 0.2081 - accuracy: 0.9133
Epoch 10/500
750/750 [==============================] - 0s 32us/sample - loss: 0.1920 - accuracy: 0.9227
Epoch 11/500
750/750 [==============================] - 0s 41us/sample - loss: 0.1842 - accuracy: 0.9307
Epoch 12/500
750/750 [==============================] - 0s 34us/sample - l

750/750 [==============================] - 0s 35us/sample - loss: 0.1257 - accuracy: 0.9440
Epoch 82/500
750/750 [==============================] - 0s 36us/sample - loss: 0.1274 - accuracy: 0.9467
Epoch 83/500
750/750 [==============================] - 0s 34us/sample - loss: 0.1230 - accuracy: 0.9507
Epoch 84/500
750/750 [==============================] - 0s 35us/sample - loss: 0.1245 - accuracy: 0.9480
Epoch 85/500
750/750 [==============================] - 0s 35us/sample - loss: 0.1257 - accuracy: 0.9507
Epoch 86/500
750/750 [==============================] - 0s 34us/sample - loss: 0.1263 - accuracy: 0.9480
Epoch 87/500
750/750 [==============================] - 0s 34us/sample - loss: 0.1253 - accuracy: 0.9453
Epoch 88/500
750/750 [==============================] - 0s 34us/sample - loss: 0.1279 - accuracy: 0.9493
Epoch 89/500
750/750 [==============================] - 0s 34us/sample - loss: 0.1261 - accuracy: 0.9493
Epoch 90/500
750/750 [==============================] - 0s 35us/samp

750/750 [==============================] - 0s 36us/sample - loss: 0.1189 - accuracy: 0.9507
Epoch 159/500
750/750 [==============================] - 0s 36us/sample - loss: 0.1226 - accuracy: 0.9493
Epoch 160/500
750/750 [==============================] - 0s 35us/sample - loss: 0.1192 - accuracy: 0.9520
Epoch 161/500
750/750 [==============================] - 0s 37us/sample - loss: 0.1265 - accuracy: 0.9467
Epoch 162/500
750/750 [==============================] - 0s 35us/sample - loss: 0.1216 - accuracy: 0.9493
Epoch 163/500
750/750 [==============================] - 0s 34us/sample - loss: 0.1209 - accuracy: 0.9453
Epoch 164/500
750/750 [==============================] - 0s 36us/sample - loss: 0.1199 - accuracy: 0.9467
Epoch 165/500
750/750 [==============================] - 0s 35us/sample - loss: 0.1198 - accuracy: 0.9520
Epoch 166/500
750/750 [==============================] - 0s 34us/sample - loss: 0.1264 - accuracy: 0.9480
Epoch 167/500
750/750 [==============================] - 0s 

KeyboardInterrupt: 